In [1217]:
import pandas as pd
import numpy as np

In [1218]:
DEBUG = False

# read in both datasets
reading_history_df   = pd.read_csv("data/db/reading_history_database.csv")
assessment_scores_df = pd.read_csv("data/db/screening_assessment_scores.csv")
print("reading history matrix original dimensions:   ", reading_history_df.shape)
print("assessment scores matrix original dimensions: ", assessment_scores_df.shape)


reading history matrix original dimensions:    (455, 34)
assessment scores matrix original dimensions:  (453, 26)


# Preprocess reading_history database

In [1219]:
# ----- shorten the column names for readability
shortened_cols = ["Q"+str(q) for q in range(1,34)]
shortened_cols.insert(0, "Participant")
reading_history_df.columns = shortened_cols

In [1220]:
# ------ delete irrelevant columns (comments entered by subjects)
del reading_history_df['Q30']
del reading_history_df['Q32']

In [1221]:
# ------- remove rows containing all NULL values
reading_history_df = reading_history_df.dropna(axis=0, how='all')

In [1222]:
# ------- find and remove any duplicate keys ('Participant')    #df[~df.name.isin(value_list)]
rd_h = reading_history_df['Participant'].value_counts()
reading_history_duplicates = rd_h[rd_h > 1]
if DEBUG: print("\nreading_history duplicate keys:")
if DEBUG: print(reading_history_duplicates.sort_index())

duplicate_participants_to_remove = reading_history_duplicates.index.tolist()
reading_history_df = reading_history_df[~reading_history_df.Participant.isin(duplicate_participants_to_remove)]

In [1223]:
# ------- remove row that has 'Participant' number as "INCOMPLETE"
reading_history_df = reading_history_df[~reading_history_df.Participant.isin(['INCOMPLETE'])]

In [1224]:
# ------- Q26 cleanup (fill in NULL values with most frequently occurring)
if DEBUG: print("BEFORE:\nNumber of NULL values in Q26: ", reading_history_df.Q26.isnull().sum())
if DEBUG: print("Frequency of values in Q26: \n", reading_history_df['Q26'].value_counts())

most_frequent_value_in_Q26 = reading_history_df['Q26'].value_counts().index[0]
reading_history_df.Q26.fillna(most_frequent_value_in_Q26, inplace=True)

if DEBUG: print("\nAFTER:\nNumber of NULL values in Q26: ", reading_history_df.Q26.isnull().sum())
if DEBUG: print("Frequency of values in Q26: \n", reading_history_df['Q26'].value_counts())

In [1225]:
# ------- Q27 cleanup (fill in NULL values with most frequently occurring)
if DEBUG: print("BEFORE:\nNumber of NULL values in Q27: ", reading_history_df.Q27.isnull().sum())
if DEBUG: print("Frequency of values in Q27: \n", reading_history_df['Q27'].value_counts())

most_frequent_value_in_Q27 = reading_history_df['Q27'].value_counts().index[0]
reading_history_df.Q27.fillna(most_frequent_value_in_Q27, inplace=True)

if DEBUG: print("\nAFTER:\nNumber of NULL values in Q27: ", reading_history_df.Q27.isnull().sum())
if DEBUG: print("Frequency of values in Q27: \n", reading_history_df['Q27'].value_counts())

In [1226]:
# ------- Q28 cleanup (fill in NULL values)
if DEBUG: print("BEFORE:\nNumber of NULL values in Q28: ", reading_history_df.Q28.isnull().sum())
if DEBUG: print("Frequency of values in Q28: \n", reading_history_df['Q28'].value_counts())
    
most_frequent_value_in_Q28 = reading_history_df['Q28'].value_counts().index[0]
reading_history_df.Q28.fillna(most_frequent_value_in_Q28, inplace=True)

if DEBUG: print("\nAFTER:\nNumber of NULL values in Q28: ", reading_history_df.Q28.isnull().sum())
if DEBUG: print("Frequency of values in Q28: \n", reading_history_df['Q28'].value_counts())

In [1227]:
# ------- Q29 cleanup (make all lowercase and strip whitespace)
if DEBUG: print("BEFORE:\nNumber of NULL values in Q29: ", reading_history_df.Q29.isnull().sum())
if DEBUG: print("Frequency of values in Q29: \n", reading_history_df['Q29'].value_counts())

# convert to lowercase and strip leading and trailing whitespace
reading_history_df['Q29'] = reading_history_df['Q29'].apply(lambda x: x.lower().strip())

# replace spaces with underscores
reading_history_df['Q29'] = reading_history_df['Q29'].apply(lambda x: '_'.join(x.split(" "))) 

if DEBUG: print("\nAFTER:\nNumber of NULL values in Q29: ", reading_history_df.Q29.isnull().sum())
if DEBUG: print("Frequency of values in Q29: \n", reading_history_df['Q29'].value_counts())

In [1228]:
# ------- Q31 cleanup (make all lowercase and strip whitespace)
if DEBUG: print("BEFORE:\nNumber of NULL values in Q31: ", reading_history_df.Q31.isnull().sum())
if DEBUG: print("Frequency of values in Q31: \n", reading_history_df['Q31'].value_counts())

# fill in NULL value with most frequent
most_frequent_value_in_Q31 = reading_history_df['Q31'].value_counts().index[0]
reading_history_df.Q31.fillna(most_frequent_value_in_Q31, inplace=True)

# convert to lowercase and strip leading and trailing whitespace
reading_history_df['Q31'] = reading_history_df['Q31'].apply(lambda x: x.lower().strip())

# collapse all 'no' responses
def process_no_responses(x):
    if x == 'no (n/a)' or x == 'no, only child':
        return('no')
    else:
        return(x)
reading_history_df['Q31'] = reading_history_df['Q31'].apply(process_no_responses) 

# replace whitespaces with underscores
reading_history_df['Q31'] = reading_history_df['Q31'].apply(lambda x: '_'.join(x.split(" ")))

if DEBUG: print("\nAFTER:\nNumber of NULL values in Q31: ", reading_history_df.Q31.isnull().sum())
if DEBUG: print("Frequency of values in Q31: \n", reading_history_df['Q31'].value_counts())

In [1229]:
# ------- Q33 cleanup
if DEBUG: print("Number of NULL values in Q33: ", reading_history_df.Q33.isnull().sum())
if DEBUG: print("\nFrequency of values in Q33: \n", reading_history_df['Q33'].value_counts())
    
# fill in NULL value with most frequent
most_frequent_value_in_Q33 = reading_history_df['Q33'].value_counts().index[0]
reading_history_df.Q33.fillna(most_frequent_value_in_Q33, inplace=True)

# lower case and strip whitespace
reading_history_df['Q33'] = reading_history_df['Q33'].apply(lambda x: x.lower().strip())

if DEBUG: print("Number of NULL values in Q33: ", reading_history_df.Q33.isnull().sum())
if DEBUG: print("\nFrequency of values in Q33: \n", reading_history_df['Q33'].value_counts())

In [1230]:
# TODO - one-hot encoding of categories

In [1231]:
# check final pre-processed reading_history dataframe
print("pre-processed reading_history dataframe dimensions: ", reading_history_df.shape)
#print("\nNULL values: \n", reading_history_df.isnull().sum())

#reading_history_df.head(50)
#print("\nNULL values: \n", reading_history_df.isnull().sum())

pre-processed reading_history dataframe dimensions:  (433, 32)


# Preprocess assessment_scores

In [1232]:
# pre-process assessment_scores
as_s = assessment_scores_df['Participant Number'].value_counts()
as_s = as_s[as_s > 1]

print("\nassesment_scores duplicate keys:")
print(as_s.sort_index())

assessment_scores_df.rename(columns={ assessment_scores_df.columns[0]: "Participant" }, inplace=True)


assesment_scores duplicate keys:
1161    2
2019    2
2234    2
4744    2
5343    2
5559    2
6970    2
7806    2
9010    2
9379    2
Name: Participant Number, dtype: int64


# Merge 2 databases

In [1233]:
# merge 2 databases on 'Participant' as key into one
merged_history_and_scores = pd.merge(reading_history_df, assessment_scores_df, how='inner', on='Participant')

In [1234]:
# reset 'Participant' as the index
merged_history_and_scores.set_index('Participant', inplace=True)
print("merged history and scores dimensions: ", merged_history_and_scores.shape)

merged history and scores dimensions:  (425, 56)


In [1235]:
# drop rows where all of the elements are NaN 
merged_history_and_scores = merged_history_and_scores.dropna(axis=0, how='all')
#print("Number of missing values (NaN) by column:")
#print(merged_history_and_scores.isnull().sum())
#print("\nDimensions after dropping empty rows:", merged_history_and_scores.shape)

In [1236]:
# Q31 cleanup: as unordered (yes, no, not sure)

In [1237]:
# Q33 cleanup: as ordered categories ("some college, etc.")
#print("all possible values in Q33: ")
#merged_history_and_scores.Q33.value_counts()